In [1]:
# 0. 데이터 전처리 

# 데이터 특징 (Data Characteristics)
# 1. 소스 및 원본 형태 (Source & Original Format)
# 정부 부처의 API 응답을 통해 수집된 정책 정보가 기본 데이터 소스입니다.
# 원본 데이터는 각 정책의 상세 정보를 담고 있는 CSV(Comma-Separated Values) 형태의 정형 데이터(structured data)입니다.

# 2. 데이터 단위 및 내용 (Granularity & Content)
# CSV의 한 행(Row)이 하나의 독립적인 정책에 대한 모든 정보를 나타냅니다.

# 각 행은 '정책명', '지원 대상 설명', '지원 내용', '신청 기간', '지역 코드' 등 필터링에 사용될 코드값과 자연어 설명이 혼재된 여러 컬럼으로 구성됩니다.

# 3. 핵심 전처리: 자연어 문서화 (Key Preprocessing: Natural Language Documentations)
# 원본 CSV를 직접 사용하지 않고, 각 행을 하나의 의미 있는 자연어 문서로 변환하는 과정을 거칩니다.

# 선별된 주요 컬럼의 내용과 코드값(예: region_code: "31")을 사람이 읽기 쉬운 문장(예: 지역: "울산광역시")으로 재구성하여 Markdown 형식의 page_content를 생성합니다.

# 이 과정은 기계 중심의 데이터를 LLM이 가장 잘 이해할 수 있는 문맥적 정보로 바꾸는 핵심 단계입니다.

# 4. 최종 데이터 구조 (Final Data Structure)
# page_content: 자연어 문서화의 결과물로 구성됩니다. 오직 시멘틱 서칭 목적으로 사용하며, context로는 전달하지 않습니다.

# metadata : 답변에 필요한 정형 데이터들을 metadata에 적재합니다. 해당 정형 데이터를 이용하여 프롬프트 단계에서 자연어로 변환 후 원문에 가까운 형태로 context를 제공합니다.



In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

os.getenv('OPENAI_API_KEY')
client = OpenAI()


In [55]:
# 0. 데이터 전처리
# 벡터 서칭이 용이하도록, 정책의 의미를 담고 있는 필드와 코드 형태로 된 값들을 자연어로 변환하여 설명문 생성 및 별개의 컬럼으로 관리 

import pandas as pd
import numpy as np

def load_maps_from_excel(filepath):
    try:
        df_codes = pd.read_excel(filepath, sheet_name='코드정보')
        code_maps = {}
        for name, group in df_codes.groupby('분류'):
            if '코드' in group.columns and '코드내용' in group.columns:
                clean_group = group.dropna(subset=['코드', '코드내용'])
                code_maps[name] = dict(zip(clean_group['코드'].astype(str), clean_group['코드내용']))
        print("✅ Excel 파일에서 코드 정보 파싱 완료!")
        return code_maps
    except Exception as e:
        print(f"🚨 Excel 파일 처리 중 오류 발생: {e}")
        return None

def create_final_document(row, code_maps):
    def get_code_name(code_type, code):
        if pd.notna(code) and code_maps:
            first_code = str(code).split(',')[0].strip()
            return code_maps.get(code_type, {}).get(first_code)
        return None

    policy_name = row.get('plcyNm', '이름 정보 없음')
    region = row.get('rgtrInstCdNm', '전국')
    if isinstance(region, str) and ('특별자치도' in region or '광역시' in region or '시' in region):
        region = region.split(' ')[0]

    category = f"{row.get('lclsfNm', '')} > {row.get('mclsfNm', '')}"
    support_content = row.get('plcySprtCn', '지원 내용 정보 없음').strip()
    
    parts = [f"이 정책은 '{region}'에서 시행하는 '{policy_name}'입니다."]
    parts.append(f"정책 분야는 '{category}'이며, '{support_content}'을 지원합니다.")

    conditions = []
    
    min_age, max_age = row.get('sprtTrgtMinAge'), row.get('sprtTrgtMaxAge')
    if pd.notna(max_age) and max_age > 0:
        if pd.notna(min_age) and min_age > 0:
            conditions.append(f"만 {int(min_age)}세에서 {int(max_age)}세 사이의 청년")
        else:
            conditions.append(f"만 {int(max_age)}세 이하의 청년")

#     if pd.notna(row.get('earnEtcCn')) and str(row.get('earnEtcCn')).strip():
#         conditions.append(f"소득 조건은 '{row['earnEtcCn']}'을 따릅니다.")
    
    # --- 수정된 자격 조건 로직 ---
    unrestricted_keywords = ['관계없음', '제한없음', '학력무관', '무관']
    
    # 결혼 상태
    mrg_name = get_code_name('mrgSttsCd', row.get('mrgSttsCd'))
    if mrg_name and mrg_name not in unrestricted_keywords:
        conditions.append(f"혼인 상태는 '{mrg_name}'이어야 합니다.")

    # 취업 상태
    job_name = get_code_name('jobCd', row.get('jobCd'))
    if job_name:
        if job_name in unrestricted_keywords:
             conditions.append("취업 상태와 관계없이 지원 가능합니다.")
        else:
             conditions.append(f"취업 상태는 '{job_name}'이어야 합니다.")
    
    # 학력
    edu_name = get_code_name('schoolCd', row.get('schoolCd'))
    if edu_name:
        if edu_name in unrestricted_keywords:
            conditions.append("학력과 관계없이 지원 가능합니다.")
        else:
            conditions.append(f"학력 조건은 '{edu_name}'입니다.")
    # --- 로직 수정 끝 ---

    if pd.notna(row.get('addAplyQlfcCndCn')) and str(row.get('addAplyQlfcCndCn')).strip():
        conditions.append(f"추가 자격: {row['addAplyQlfcCndCn']}")

    if conditions:
        parts.append("지원 대상은 " + ", ".join(filter(None, conditions)) + "입니다.")
    
    if pd.notna(row.get('plcyAplyMthdCn')) and str(row.get('plcyAplyMthdCn')).strip():
        parts.append(f"신청 방법은 {row['plcyAplyMthdCn']}입니다.")

    return " ".join(parts)

In [56]:
if __name__ == '__main__':
    POLICY_CSV_PATH = './policy_data.csv'
    CODE_EXCEL_PATH = './code_table.xlsx'
    OUTPUT_CSV_PATH = './policies_with_documents_final.csv'

    try:
        df_raw = pd.read_csv(POLICY_CSV_PATH, encoding='utf-8')
        print(f"✅ 원본 CSV 데이터 '{POLICY_CSV_PATH}' 로딩 성공!")
    except FileNotFoundError:
        print(f"🚨 오류: '{POLICY_CSV_PATH}' 파일을 찾을 수 없습니다.")
        exit()

    df_raw['sprtTrgtMinAge'] = pd.to_numeric(df_raw['sprtTrgtMinAge'], errors='coerce')
    df_raw['sprtTrgtMaxAge'] = pd.to_numeric(df_raw['sprtTrgtMaxAge'], errors='coerce')

    code_maps = load_maps_from_excel(CODE_EXCEL_PATH)

    if code_maps:
        print("\n문서 생성을 시작합니다...")
        df_raw['document'] = df_raw.apply(lambda row: create_final_document(row, code_maps), axis=1)
        print("✅ 최종 자연어 설명문 생성 완료!")
        df_raw.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
        print(f"\n✅ 모든 문서가 포함된 최종 결과가 '{OUTPUT_CSV_PATH}' 파일로 저장되었습니다.")
    else:
        print("\n\n🚨 코드맵 로딩 실패로 전체 프로세스를 중단합니다.")

✅ 원본 CSV 데이터 './policy_data.csv' 로딩 성공!
✅ Excel 파일에서 코드 정보 파싱 완료!

문서 생성을 시작합니다...
✅ 최종 자연어 설명문 생성 완료!

✅ 모든 문서가 포함된 최종 결과가 './policies_with_documents_final.csv' 파일로 저장되었습니다.


In [57]:
# 1. 인덱싱

# 인덱싱 전략
# a. 청킹은 하지 말자. 
# => 1) 문서가 정책이라는 하나의 주제 안에서 잘 정제된 형태. 정책의 범위 바깥에서 청킹된 각 정보는 유의미할까? 정책이라는 틀이 없이 지원 조건, 정책 설명 내용 등이 유의미한가? X.
# 2) 임베딩 모델이 허용하는 토큰 사이즈인 8K를 초과하지 않으므로, 토큰 문제도 없음.
# b. 어떤 정보를 임베딩 할 것인가? 
# => 의미적으로 유의미한 컬럼을 추출한다. 이후 컬럼 이름과 내용을 LLM이 이해하기 쉽도록 자연어로 변환하여 별도의 컬럼으로 다룬다. 이 컬럼을 임베딩 및 page content로 다룬다.
# c. 이 page_content는 시멘틱 서칭용으로만 사용한다. context에는 metadata를 활용하여 원문에 가깝게 제공한다.

# 사용할 컬럼 정하기

'''
plcyNo	정책번호
plcyPvsnMthdCd	정책제공방법코드
plcyNm	정책명
plcyKywdNm	정책키워드명
plcyExplnCn	정책설명내용
lclsfNm	정책대분류명
mclsfNm	정책중분류명
plcySprtCn	정책지원내용
sprvsnInstCdNm	주관기관코드명
operInstCdNm	운영기관코드명
operInstPicNm	운영기관담당자명
sprtSclLmtYn	지원규모제한여부
aplyPrdSeCd	신청기간구분코드
bizPrdSeCd	사업기간구분코드
bizPrdBgngYmd	사업기간시작일자
bizPrdEndYmd	사업기간종료일자
bizPrdEtcCn	사업기간기타내용
plcyAplyMthdCn	정책신청방법내용
srngMthdCn	심사방법내용
aplyUrlAddr	신청URL주소
sbmsnDcmntCn	제출서류내용
etcMttrCn	기타사항내용
refUrlAddr1	참고URL주소1
refUrlAddr2	참고URL주소2
sprtTrgtMinAge	지원대상최소연령
sprtTrgtMaxAge	지원대상최대연령
sprtTrgtAgeLmtYn	지원대상연령제한여부
mrgSttsCd	결혼상태코드
earnMinAmt	소득최소금액
earnMaxAmt	소득최대금액
earnEtcCn	소득기타내용
addAplyQlfcCndCn	추가신청자격조건내용
ptcpPrpTrgtCn	참여제안대상내용
zipCd	정책거주지역코드
plcyMajorCd	정책전공요건코드
jobCd	정책취업요건코드
schoolCd	정책학력요건코드
aplyYmd	신청기간
frstRegDt	최초등록일시
lastMdfcnDt	최종수정일시
sbizCd	정책특화요건코드
''' 



'\nplcyNo\t정책번호\nplcyPvsnMthdCd\t정책제공방법코드\nplcyNm\t정책명\nplcyKywdNm\t정책키워드명\nplcyExplnCn\t정책설명내용\nlclsfNm\t정책대분류명\nmclsfNm\t정책중분류명\nplcySprtCn\t정책지원내용\nsprvsnInstCdNm\t주관기관코드명\noperInstCdNm\t운영기관코드명\noperInstPicNm\t운영기관담당자명\nsprtSclLmtYn\t지원규모제한여부\naplyPrdSeCd\t신청기간구분코드\nbizPrdSeCd\t사업기간구분코드\nbizPrdBgngYmd\t사업기간시작일자\nbizPrdEndYmd\t사업기간종료일자\nbizPrdEtcCn\t사업기간기타내용\nplcyAplyMthdCn\t정책신청방법내용\nsrngMthdCn\t심사방법내용\naplyUrlAddr\t신청URL주소\nsbmsnDcmntCn\t제출서류내용\netcMttrCn\t기타사항내용\nrefUrlAddr1\t참고URL주소1\nrefUrlAddr2\t참고URL주소2\nsprtTrgtMinAge\t지원대상최소연령\nsprtTrgtMaxAge\t지원대상최대연령\nsprtTrgtAgeLmtYn\t지원대상연령제한여부\nmrgSttsCd\t결혼상태코드\nearnMinAmt\t소득최소금액\nearnMaxAmt\t소득최대금액\nearnEtcCn\t소득기타내용\naddAplyQlfcCndCn\t추가신청자격조건내용\nptcpPrpTrgtCn\t참여제안대상내용\nzipCd\t정책거주지역코드\nplcyMajorCd\t정책전공요건코드\njobCd\t정책취업요건코드\nschoolCd\t정책학력요건코드\naplyYmd\t신청기간\nfrstRegDt\t최초등록일시\nlastMdfcnDt\t최종수정일시\nsbizCd\t정책특화요건코드\n'

In [58]:
# 도큐먼트로 만들기

import csv
from langchain_core.documents import Document

def create_documents_from_csv(csv_file_path: str) -> list[Document]:
    """
    CSV 파일에서 정책 정보를 읽어와
    LangChain의 Document 객체 리스트로 변환합니다.

    - 각 row는 하나의 정책 데이터를 나타냅니다.
    - 서술형 정보는 page_content로 조합합니다.
    - 정형 정보는 metadata로 저장합니다.
    """
    
    documents = []
    
    try:
        # UTF-8 인코딩으로 CSV 파일을 엽니다.
        with open(csv_file_path, mode='r', encoding='utf-8-sig') as csvfile:
            # 각 row를 딕셔너리 형태로 읽어옵니다.
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # 1. page_content 구성: 검색의 대상이 될 자연어 텍스트
                page_content = row.get('document')

                # 2. metadata 구성: 필터링 및 출처 표시에 사용할 정형 데이터
                # 항상 원본값 그대로
                metadata = {
                    # 정책 기본 정보
                    "plcyNo": row.get('plcyNo', '정보 없음'),
                    "plcyNm": row.get('plcyNm', '정보 없음'),
                    "plcyKywdNm": row.get('plcyKywdNm', '정보 없음'),
                    "plcyExplnCn": row.get('plcyExplnCn', '정보 없음'),
                    "lclsfNm": row.get('lclsfNm', '정보 없음'),
                    "mclsfNm": row.get('mclsfNm', '정보 없음'),
                    "plcySprtCn": row.get('plcySprtCn', '정보 없음'),
                    "plcyPvsnMthdCd": row.get('plcyPvsnMthdCd', '정보 없음'),

                    # 기관 정보
                    "rgtrInstCdNm": row.get('rgtrInstCdNm', '정보 없음'),
                    

                    # 기간 정보
                    "aplyPrdSeCd": row.get('aplyPrdSeCd', '정보 없음'),
                    "bizPrdSeCd": row.get('bizPrdSeCd', '정보 없음'),
                    "bizPrdBgngYmd": row.get('bizPrdBgngYmd', '정보 없음'),
                    "bizPrdEndYmd": row.get('bizPrdEndYmd', '정보 없음'),
                    "bizPrdEtcCn": row.get('bizPrdEtcCn', '정보 없음'),
                    "aplyYmd": row.get('aplyYmd', '정보 없음'),
                    "frstRegDt": row.get('frstRegDt', '정보 없음'),
                    "lastMdfcnDt": row.get('lastMdfcnDt', '정보 없음'),

                    # 신청 및 방법
                    "plcyAplyMthdCn": row.get('plcyAplyMthdCn', '정보 없음'),
                    "srngMthdCn": row.get('srngMthdCn', '정보 없음'),
                    "sbmsnDcmntCn": row.get('sbmsnDcmntCn', '정보 없음'),
                    "aplyUrlAddr": row.get('aplyUrlAddr', '정보 없음'),

                    # 지원 조건
                    "sprtSclLmtYn": row.get('sprtSclLmtYn', '정보 없음'),
                    "sprtTrgtMinAge": row.get('sprtTrgtMinAge', '정보 없음'),
                    "sprtTrgtMaxAge": row.get('sprtTrgtMaxAge', '정보 없음'),
                    "sprtTrgtAgeLmtYn": row.get('sprtTrgtAgeLmtYn', '정보 없음'),
                    "mrgSttsCd": row.get('mrgSttsCd', '정보 없음'),
                    "earnMinAmt": row.get('earnMinAmt', '정보 없음'),
                    "earnMaxAmt": row.get('earnMaxAmt', '정보 없음'),
                    "earnEtcCn": row.get('earnEtcCn', '정보 없음'),
                    "addAplyQlfcCndCn": row.get('addAplyQlfcCndCn', '정보 없음'),
                    "ptcpPrpTrgtCn": row.get('ptcpPrpTrgtCn', '정보 없음'),

                    # 요건 코드(target, 대상)
                    "zipCd": row.get('zipCd', '정보 없음'),
                    "plcyMajorCd": row.get('plcyMajorCd', '정보 없음'),
                    "jobCd": row.get('jobCd', '정보 없음'),
                    "schoolCd": row.get('schoolCd', '정보 없음'),
                    "sbizCd": row.get('sbizCd', '정보 없음'),

                    # 기타
                    "etcMttrCn": row.get('etcMttrCn', '정보 없음'),
                    "refUrlAddr1": row.get('refUrlAddr1', '정보 없음'),
                    "refUrlAddr2": row.get('refUrlAddr2', '정보 없음'),
                    
                }

                
                documents.append(Document(page_content=page_content.strip(), metadata=metadata))
                
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        
    return documents


In [59]:
docs = create_documents_from_csv('./policies_with_documents_final.csv')

In [60]:


# 임베딩
import os 
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.documents import Document
load_dotenv()

# 임베딩 모델 및 Chroma DB 준비
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.getenv("OPENAI_API_KEY"))
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=os.getenv("OPENAI_API_KEY"))

# Chroma DB 준비
vectorstore = Chroma(
    collection_name="policy_collection_augmented_summary_added_openai_large",
    embedding_function=embedding_model,
    persist_directory="./chroma_db_policy"
)


In [61]:
import tiktoken
from langchain_core.documents import Document

def count_tokens(text, model="text-embedding-3-large"):
    # OpenAI 임베딩 모델들은 cl100k_base 인코딩 사용
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))


avg = 0;
for i in docs:
    token_count = count_tokens(i.page_content)
    avg += token_count
    if token_count >= 1000:
        print(f"토큰 수: {token_count}")
print(f"총 토큰 수: {avg}")



토큰 수: 1517
토큰 수: 1291
토큰 수: 1048
토큰 수: 2225
토큰 수: 1125
토큰 수: 1116
토큰 수: 1078
토큰 수: 1338
토큰 수: 1147
토큰 수: 1264
토큰 수: 1165
토큰 수: 1092
토큰 수: 1147
토큰 수: 1062
토큰 수: 1586
토큰 수: 1148
토큰 수: 1417
토큰 수: 1301
토큰 수: 1734
토큰 수: 1147
토큰 수: 1014
토큰 수: 1463
토큰 수: 1367
토큰 수: 1092
토큰 수: 1099
토큰 수: 1372
토큰 수: 1078
토큰 수: 1246
토큰 수: 1042
토큰 수: 1120
토큰 수: 1470
토큰 수: 1769
토큰 수: 1929
토큰 수: 1509
토큰 수: 1008
토큰 수: 1178
토큰 수: 1064
토큰 수: 1231
토큰 수: 1159
토큰 수: 1036
토큰 수: 1061
토큰 수: 1635
토큰 수: 1073
토큰 수: 1223
토큰 수: 1305
토큰 수: 1440
토큰 수: 1363
토큰 수: 1188
토큰 수: 1342
토큰 수: 1339
토큰 수: 1217
토큰 수: 1041
토큰 수: 1217
토큰 수: 1263
토큰 수: 1120
토큰 수: 1045
토큰 수: 1500
토큰 수: 1465
토큰 수: 1053
토큰 수: 1724
토큰 수: 1295
토큰 수: 1090
토큰 수: 1117
토큰 수: 1421
토큰 수: 1056
토큰 수: 1161
토큰 수: 1632
토큰 수: 1587
토큰 수: 1333
토큰 수: 1019
토큰 수: 1168
토큰 수: 2674
토큰 수: 1004
토큰 수: 1100
토큰 수: 1104
토큰 수: 1285
토큰 수: 1295
토큰 수: 1097
토큰 수: 1363
토큰 수: 1330
토큰 수: 1059
토큰 수: 1094
토큰 수: 1107
토큰 수: 1058
토큰 수: 1118
토큰 수: 1480
토큰 수: 2505
토큰 수: 1034
토큰 수: 1348
토큰 수: 1164
토큰 수: 1088

# ------------------ 준비 과정 끝 ----------------------

In [5]:
# 1. 파서 함수 : 사용자의 질문을 받아 JSON 스키마에 맞춰 필터를 생성하는 함수.

import os
import json
from openai import OpenAI

def create_filter_from_query(client: OpenAI, user_query: str) -> dict:
    """
    사용자의 자연어 질문을 분석하고,
    정책 필터링에 사용할 구조화된 JSON(파이썬 딕셔너리)을 생성합니다.

    Args:
        client: 초기화된 OpenAI 클라이언트 객체
        user_query: 사용자의 원본 질문 문자열

    Returns:
        추출된 필터 정보가 담긴 딕셔너리
    """
    # LLM에게 전달할 시스템 프롬프트. 역할, 지시사항, 허용 값, JSON 스키마를 명시합니다.
    system_prompt = """
# ROLE
You are an expert at extracting key information for filtering South Korean youth policies from a user's query.

# INSTRUCTION
- Analyze the user's query and generate a JSON object that strictly follows the provided `JSON SCHEMA`.
- CRITICAL: When extracting regions, you MUST normalize them to their full official administrative names. (e.g., "서울", "서울시" -> "서울특별시" / "경기" -> "경기도" / "부산" -> "부산광역시" / "성남" -> "성남시" / "종로" -> "종로구")
- For fields with `ALLOWED VALUES`, you MUST choose from the provided list. If a user's term is a synonym, map it to the correct value (e.g., "실업자" -> "미취업").
- If a value is not mentioned, use `null` for single values or an empty list `[]` for array values.
- Do NOT make up values that are not in the `ALLOWED VALUES` list.
- Output ONLY the JSON object.

# ALLOWED VALUES
- 'job_status': ["재직자", "자영업자", "미취업자", "프리랜서", "일용근로자",
"(예비)창업자", "단기근로자", "영농종사자", "기타", "제한없음"]
- 'marriage_status': ["기혼", "미혼", "제한없음"]
- 'education_levels': ["고졸 미만", "고교 재학", "고졸 예정", "고교 졸업", "대학 재학", "대졸 예정",
"대학 졸업", "석·박사", "기타", "제한없음"]
- 'majors': ["인문계열", "사회계열", "상경계열", "이학계열", "공학계열", 
"예체능계열", "농산업계열", "기타", "제한없음"]
- 'categories': ["일자리", "주거", "교육", "복지문화", "참여권리"]
- 'subcategories': ["취업", "재직자", "창업", "주택 및 거주지", "기숙사",
"전월세 및 주거급여 지원", "미래역량강화", "교육비지원", "온라인교육", "취약계층 및 금융지원",
"건강", "예술인지원", "문화활동", "청년참여", "정책인프라구축", "청년국제교류", "권익보호"]
- 'specializations': ["중소기업", "여성", "기초생활수급자", "한부모가정", "장애인",
"농업인", "군인", "지역인재", "기타", "제한없음"]
- 'keywords': ["대출", "보조금", "바우처", "금리혜택", "교육지원", "맞춤형상담서비스",
"인턴", "벤처", "중소기업", "청년가장", "장기미취업청년", "공공임대주택",
"신용회복", "육아", "출산", "해외진출", "주거지원"]


# JSON SCHEMA
{
  "age": "number | null",
  "income": "number | null",
  "regions": ["string"],
  "job_status": ["string"],
  "marriage_status": "string | null",
  "education_levels": ["string"],
  "majors": ["string"],
  "categories": ["string"],
  "subcategories": ["string"],
  "specializations": ["string"],
  "keywords": ["string"]
}

# EXAMPLES
---
user_query: "서울 사는 25세 미취업자인데, 창업 지원금 좀 알아봐줘"
{
  "age": 25,
  "income": null,
  "regions": ["서울특별시"],
  "job_status": ["미취업자", "(예비)창업자"],
  "marriage_status": null,
  "education_levels": [],
  "majors": [],
  "categories": ["일자리"],
  "subcategories": ["창업"],
  "specializations": [],
  "keywords": ["보조금", "벤처"]
}
---
user_query: "강원 춘천에 거주하는 고졸 학력으로 지원 가능한 주거 대출 정책 있어?"
{
  "age": null,
  "income": null,
  "regions": ["강원특별자치도", "춘천시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["고교 졸업"],
  "majors": [],
  "categories": ["주거"],
  "subcategories": ["주택 및 거주지", "기숙사", "전월세 및 주거급여 지원"],
  "specializations": [],
  "keywords": ["대출"]
}
---
user_query: "목포에 사는 사람인데 석사 지원 정책같은거 있냐"
{
  "age": null,
  "income": null,
  "regions": ["전라남도", "목포시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["석·박사"],
  "majors": [],
  "categories": [],
  "subcategories": [],
  "specializations": [],
  "keywords": []
}

---
user_query: "전국 단위로 지원해주는 청년 창업 정책 알려줘"
{
  "age": null,
  "income": null,
  "regions": [],
  "job_status": ["(예비)창업자"],
  "marriage_status": null,
  "education_levels": [],
  "majors": [],
  "categories": ["일자리"],
  "subcategories": ["창업"],
  "specializations": [],
  "keywords": []
}
"""

    try:
        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o", 
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )
        
        # 반환된 JSON 문자열을 파이썬 딕셔너리로 파싱
        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

In [6]:
# 후보 정책들 필터링

import mysql.connector
from mysql.connector import Error
def _get_all_related_region_codes(cursor, region_names: list) -> list:
    """
    지역명 리스트를 받아 관련된 모든 지역 코드(시/도 및 하위 시/군/구)를 반환합니다.
    """
    if not region_names:
        return []
    all_codes = set()
    region_regex = '|'.join(region_names)
    
    query = "SELECT code FROM region_codes WHERE sido REGEXP %s OR sigungu REGEXP %s"
    cursor.execute(query, (region_regex, region_regex))
    results = cursor.fetchall()
    for row in results:
        all_codes.add(row[0])
    
    return list(all_codes)

def get_rdb_candidate_ids(db_connection, filters: dict) -> list:
    """
    [최소 조건 버전] 기간과 지역 필터만을 사용하여 RDB에서 1차 후보군을 조회합니다.
    """
    try:
        cursor = db_connection.cursor()

        base_query = "SELECT DISTINCT p.policy_id FROM policies p"
        joins = set()
        where_conditions = []
        params = []

        # 1. 신청 기간 필터
        application_date_condition = """
        (p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))
        """
        where_conditions.append(application_date_condition.strip())
        
        # 2. 사업 기간 필터
        biz_date_condition = "(p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))"
        where_conditions.append(biz_date_condition)

        # 3. 지역 필터
        if filters.get("regions"):
            region_codes = _get_all_related_region_codes(cursor, filters["regions"])
            if region_codes:
                joins.add("JOIN policy_regions pr ON p.policy_id = pr.policy_id")
                placeholders = ', '.join(['%s'] * len(region_codes))
                where_conditions.append(f"pr.region_code IN ({placeholders})")
                params.extend(region_codes)
        
        # 최종 쿼리 조립
        final_query = base_query
        if joins:
            final_query += " " + " ".join(list(joins))
        if where_conditions:
            final_query += " WHERE (" + ") AND (".join(where_conditions) + ")"
        final_query += ";"

        # 디버깅을 위해 쿼리 템플릿과 파라미터를 별도로 출력
        print("--- Generated SQL Query (Simplified) ---")
        print(final_query)
        print("\n--- SQL Parameters ---")
        print(params)

        # 쿼리 실행
        cursor.execute(final_query, params)
        candidate_ids = [item[0] for item in cursor.fetchall()]
        return candidate_ids

    except Error as e:
        print(f"Database error: {e}")
        return []
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()

In [7]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

def semantic_search(
    candidate_ids: list,
    original_query: str,
    extracted_filters: dict,
    vdb_directory: str,
    k: int = 5,
    fetch_k: int = 20,
    lambda_mult: float = 1.0
) -> list:
    '''
    전달받은 필터를 통해 사용자의 질문을 증강하고, 후보 인덱스 내에서만 R을 수행하여 검색 정확도를 높여 시멘틱 서칭을 수행하는 함수입니다.
    Args:
        candidate_ids (list): 검색할 후보 ID 목록입니다.
        original_query (str): 원본 검색 쿼리 문자열입니다.
        extracted_filters (dict): 추출된 필터 정보 딕셔너리입니다.
        vdb_directory (str): 벡터 데이터베이스 디렉토리 경로입니다.
        k (int, optional): 반환할 상위 검색 결과의 개수입니다. 기본값은 5입니다.
        fetch_k (int, optional): 유사도 검색을 위해 가져올 초기 결과의 개수입니다. 기본값은 20입니다.
        lambda_mult (float, optional): 재정렬(reranking) 시 사용되는 람다 값입니다. 기본값은 0.7입니다.
    '''
    if not candidate_ids:
        return []

    # 1. 보강된 검색어 생성
    boost_keywords = []
    # soft_filter_keys = ["job_status", "education_levels", "keywords", "categories"]
    soft_filter_keys = []
    for key in soft_filter_keys:
        if extracted_filters.get(key):
            boost_keywords.extend(extracted_filters[key])
    
    synthetic_query = original_query + " " + " ".join(list(set(boost_keywords)))
    print(f"\n--- Generated Vector Search Query ---\n{synthetic_query}\n")

    # 2. 임베딩 모델 및 ChromaDB 로드
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = Chroma(
        collection_name="policy_collection_summary_added_openai_large", 
        
        embedding_function=embedding_model,
        persist_directory=vdb_directory
    )

    # 3. 필터가 적용된 Retriever 생성
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": k,
            "fetch_k": fetch_k,
            "lambda_mult": lambda_mult,
            "filter": {'plcyNo': {'$in': candidate_ids}}
        }
    )

    # 4. Retriever 실행 및 Document 리스트 반환
    docs = retriever.invoke(synthetic_query)
    
    return docs

In [8]:
# 필요한 모든 함수와 라이브러리를 임포트했다고 가정합니다.
# from step1_parser import create_filter_from_query
# from step2_rdb_filter import get_rdb_candidate_ids
# from step3_semantic_search import semantic_search
from openai import OpenAI
import mysql.connector

# --- 0. 기본 설정 ---
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
VDB_DIRECTORY = "./chroma_db_policy"
openai_client = OpenAI()

# --- 1. 사용자 질문 ---
# user_query = "구로구에 사는데 주거지원 정책좀 알려줘."
# user_query = "대구 웹툰 캠퍼스"
user_query = "웹툰캠퍼스 운영 및 인력 양성 정책에 대해 알려줘."
# user_query = "웹툰 작가가 되고 싶은데, 교육이나 지원금 받을 수 있는 프로그램 찾아줘."
# user_query = "벤처 지원이랑 맞춤형 상담 서비스를 동시에 해주는 문화 정책이 궁금해."

print(f"--- User Query ---\n{user_query}\n")


# --- 2. 1단계: LLM 파서 실행 ---
llm_filters = create_filter_from_query(openai_client, user_query)
print(f"--- Step 1: LLM Filter Results ---\n{llm_filters}\n")
# --- 3. 2단계: RDB 필터링 실행 ---
try:
    connection = mysql.connector.connect(
        host='localhost',
        database='toyprj4',
        user='root',
        password='1234'
    )
    rdb_candidate_ids = get_rdb_candidate_ids(connection, llm_filters)
    print(f"--- Step 2: RDB Candidate IDs ---\n{rdb_candidate_ids}\n")

finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()


# --- 4. 3단계: Vector DB 랭킹 실행 ---
final_documents = semantic_search(
    candidate_ids=rdb_candidate_ids,
    original_query=user_query,
    extracted_filters=llm_filters,
    vdb_directory=VDB_DIRECTORY
)


# --- 5. 최종 결과 확인 ---
print("\n--- Step 3: Final Retrieved Documents ---")
for doc in final_documents:
    print(f"Policy ID: {doc.metadata.get('plcyNo')}")
    print(f"application date: {doc.metadata.get('aplyYmd')}")
    print(f"Content: {doc.page_content}...") # 내용 일부만 출력
    print("-" * 20)

--- User Query ---
웹툰캠퍼스 운영 및 인력 양성 정책에 대해 알려줘.

--- Step 1: LLM Filter Results ---
{'age': None, 'income': None, 'regions': [], 'job_status': [], 'marriage_status': None, 'education_levels': [], 'majors': ['예체능계열'], 'categories': ['교육'], 'subcategories': ['미래역량강화'], 'specializations': [], 'keywords': []}

--- Generated SQL Query (Simplified) ---
SELECT DISTINCT p.policy_id FROM policies p WHERE ((p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))) AND ((p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date)));

--- SQL Parameters ---
[]
--- Step 2: RDB Candidate IDs ---
['20250717005400211358', '20250717005400211356', '20250717005400211355', '20250717005400211354', '20250717005400211353', '20250717005400211352', '20250717005400211350', '20250717005400211349', '20250717005400211348', '20250717005400211347', '20250717005400211346', '20250717005400211345', '20250717005400211343', '2025071

In [17]:
final_documents[0].metadata

{'sprtSclLmtYn': 'N',
 'srngMthdCn': '',
 'plcyKywdNm': '교육지원',
 'earnEtcCn': '',
 'plcyNo': '20250716005400211297',
 'schoolCd': '0049010',
 'earnMaxAmt': '0.0',
 'refUrlAddr2': '',
 'plcyExplnCn': '만화·웹툰 작가, 예비작가를 위한 공간 및 프로그램 운영으로 지역 만화산업 생태계 육성',
 'zipCd': '31110,31140,31170,31200,31710',
 'etcMttrCn': '',
 'sprvsnInstCdNm': '울산광역시',
 'sprtTrgtAgeLmtYn': 'N',
 'bizPrdEtcCn': '',
 'bizPrdSeCd': '56001.0',
 'aplyYmd': '',
 'plcyMajorCd': '0011009',
 'earnMinAmt': '0.0',
 'aplyUrlAddr': '',
 'lastMdfcnDt': '2025-07-16 17:25:50',
 'plcySprtCn': '○ 사업기간 : ’24. 4.~12. \n\n○ 위    치 : 중구 종가로 406-21 울산비즈파크 6층  \n\n○ 지원대상 : 만화ㆍ웹툰 작가, 웹툰산업 분야 진로 청년 등 \n\n○ 지원내용 : 웹툰 작가 등단을 위한 작품제작 지원(교육‧멘토링) 등\n \n○ 수행기관 : 울산정보산업진흥원',
 'plcyNm': '울산웹툰캠퍼스 운영',
 'operInstCdNm': '재단법인울산정보산업진흥원',
 'aplyPrdSeCd': '57002',
 'jobCd': '0013010',
 'bizPrdBgngYmd': '20250401',
 'plcyPvsnMthdCd': '42002.0',
 'refUrlAddr1': 'https://www.uipa.or.kr/',
 'ptcpPrpTrgtCn': '',
 'sbmsnDcmntCn': '',
 'mrgSttsCd': '55003.0',
 'l

In [44]:
# 문서 결합 체인은 어떤걸 쓸까?
# 1. stuff
# 2. map-reduce
# 3. refine
# 4. map-rerank

# 사실 다 써보고 싶다.
# stuff는 가장 보편적으로 사용되면서도 높은 성능을 보이는 체인..
# map-reduce는 대량 문서 처리에 용이하고
# refine은 하나의 보고서로 만드는거
# map-rerank는 최고의 답안 한개를 추출하는거..

In [9]:
# metadata to context 포맷팅 함수
import pandas as pd
from functools import partial
# from langchain_core.documents import Document # 실제 LangChain 환경에서 사용
# from langchain_core.runnables import RunnableParallel, RunnablePassthrough # 실제 LCEL 구성 시 사용

# =================================================================
# 1. 엑셀 파일 로드 및 파싱하여 CODE_TABLE_MAP 생성
# =================================================================
CODE_TABLE_MAP = {}
excel_file_path = 'code_table.xlsx' # .py 파일과 같은 위치에 있는 엑셀 파일명
sheet_name_to_read = '코드정보'     # 읽어올 시트 이름

try:
    # 엑셀 파일의 '코드정보' 시트에서 데이터를 읽어옵니다.
    df_codes = pd.read_excel(excel_file_path, sheet_name=sheet_name_to_read)

    # '분류' 열의 비어있는 값을 바로 위의 값으로 채웁니다 (엑셀의 병합된 셀 처리).
    df_codes['분류'] = df_codes['분류'].ffill()

    # 데이터프레임을 순회하며 코드 맵을 구성합니다.
    for _, row in df_codes.iterrows():
        category = row['분류']
        # 코드 값은 문자열로 변환하여 일관성을 유지합니다.
        code = str(row['코드'])
        value = row['코드내용']
        
        if category not in CODE_TABLE_MAP:
            CODE_TABLE_MAP[category] = {}
        CODE_TABLE_MAP[category][code] = value
    print(f"✅ 엑셀 파일('{excel_file_path}')을 성공적으로 로드하고 파싱했습니다.")

except FileNotFoundError:
    print(f"⚠️ 경고: '{excel_file_path}' 파일을 찾을 수 없습니다. 코드 변환이 비활성화됩니다.")
except Exception as e:
    print(f"🚨 오류: 엑셀 파일 처리 중 오류 발생: {e}")


# =================================================================
# 2. 모든 필드를 변환하는 최종 포맷팅 함수 (안정성 개선 버전)
# =================================================================
def format_docs(docs: list, code_map: dict) -> str:
    """
    (최종본) Retriever의 Document 리스트와 코드 테이블을 받아,
    모든 metadata 필드를 활용하여 LLM이 이해하기 좋은 상세한 Markdown 형식으로 변환합니다.
    값이 없는 필드는 출력하지 않아 안정성을 높였습니다.
    """
    
    # --- Helper Functions ---
    def get_code_value(category: str, code) -> str:
        """코드 테이블(code_map)을 조회하여 코드에 해당하는 명칭을 반환합니다."""
        if not category or pd.isna(code) or code == '': return ""
        code_str = str(code)
        if code_str.endswith('.0'): code_str = code_str[:-2]
        return code_map.get(category, {}).get(code_str, f"코드({code_str})")

    def format_date(date_str: str) -> str:
        """'YYYYMMDD' 형식의 문자열을 'YYYY-MM-DD'로 변환합니다."""
        if date_str and isinstance(date_str, str) and len(date_str) == 8:
            return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return date_str

    def format_bool(bool_str: str) -> str:
        """'Y'는 '예', 'N'은 '아니오'로 변환합니다."""
        if bool_str == 'Y': return '예'
        if bool_str == 'N': return '아니오'
        return ''

    # --- Main Loop ---
    formatted_strings = []
    for i, doc in enumerate(docs):
        meta = doc.metadata
        doc_string = f"--- [문서 {i+1}: {meta.get('plcyNm', '제목 없음')}] ---\n"
        
        # 각 섹션의 내용을 동적으로 구성
        sections = {}

        # 기본 정보
        basic_info = []
        if meta.get('plcyNo'): basic_info.append(f"- **정책 ID**: {meta.get('plcyNo')}")
        if meta.get('plcyNm'): basic_info.append(f"- **정책명**: {meta.get('plcyNm')}")
        if meta.get('plcyExplnCn'): basic_info.append(f"- **정책 요약**: {meta.get('plcyExplnCn')}")
        if meta.get('lclsfNm') and meta.get('mclsfNm'): basic_info.append(f"- **분류**: {meta.get('lclsfNm')} > {meta.get('mclsfNm')}")
        if meta.get('plcyKywdNm'): basic_info.append(f"- **정책 키워드**: {meta.get('plcyKywdNm')}")
        if basic_info: sections["기본 정보"] = basic_info
        
        # 지원 내용
        support_info = []
        if meta.get('plcySprtCn'): support_info.append(f"- **상세 지원 내용**: \n{meta.get('plcySprtCn')}")
        if get_code_value('plcyPvsnMthdCd', meta.get('plcyPvsnMthdCd')): support_info.append(f"- **지원 방식**: {get_code_value('plcyPvsnMthdCd', meta.get('plcyPvsnMthdCd'))}")
        if format_bool(meta.get('sprtSclLmtYn')): support_info.append(f"- **지원 규모 제한 여부**: {format_bool(meta.get('sprtSclLmtYn'))}")
        if support_info: sections["지원 내용"] = support_info

        # 신청 및 기간
        apply_info = []
        if meta.get('bizPrdBgngYmd') or meta.get('bizPrdEndYmd'): apply_info.append(f"- **사업 기간**: {format_date(meta.get('bizPrdBgngYmd'))} ~ {format_date(meta.get('bizPrdEndYmd'))}")
        if meta.get('bizPrdEtcCn'): apply_info.append(f"- **사업 기간 설명**: {meta.get('bizPrdEtcCn')}")
        if meta.get('aplyYmd'): apply_info.append(f"- **지원 기간**: {meta.get('aplyYmd')}")
        if meta.get('plcyAplyMthdCn'): apply_info.append(f"- **신청 방법**: {meta.get('plcyAplyMthdCn')}")
        if meta.get('aplyUrlAddr'): apply_info.append(f"- **신청 사이트**: {meta.get('aplyUrlAddr')}")
        if meta.get('sbmsnDcmntCn'): apply_info.append(f"- **제출 서류**: {meta.get('sbmsnDcmntCn')}")
        if apply_info: sections["신청 및 기간"] = apply_info

        # 지원 대상 조건
        target_info = []
        if meta.get('sprtTrgtAgeLmtYn') == 'Y' and (meta.get('sprtTrgtMinAge') or meta.get('sprtTrgtMaxAge')): target_info.append(f"- **연령**: 만 {int(float(meta.get('sprtTrgtMinAge', 0)))}세 ~ 만 {int(float(meta.get('sprtTrgtMaxAge', 0)))}세")
        # if meta.get('zipCd'): target_info.append(f"- **거주지**: {meta.get('zipCd')}")
        if get_code_value('schoolCd', meta.get('schoolCd')): target_info.append(f"- **학력**: {get_code_value('schoolCd', meta.get('schoolCd'))}")
        if get_code_value('mrgSttsCd', meta.get('mrgSttsCd')): target_info.append(f"- **혼인 상태**: {get_code_value('mrgSttsCd', meta.get('mrgSttsCd'))}")
        if get_code_value('jobCd', meta.get('jobCd')): target_info.append(f"- **직업 상태**: {get_code_value('jobCd', meta.get('jobCd'))}")
        if get_code_value('sbizCd', meta.get('sbizCd')): target_info.append(f"- **특화 분야**: {get_code_value('sbizCd', meta.get('sbizCd'))}")
        
        # if meta.get('earnMinAmt') or meta.get('earnMaxAmt'): target_info.append(f"- **소득 조건**: 최저 {meta.get('earnMinAmt', '0')}원 ~ 최고 {meta.get('earnMaxAmt', '0')}원")
        
        min_earn_str = meta.get('earnMinAmt', '0.0')
        max_earn_str = meta.get('earnMaxAmt', '0.0')
        min_earn = float(min_earn_str)
        max_earn = float(max_earn_str)
        if min_earn == 0.0 and max_earn == 0.0:
            target_info.append("- **소득 조건**: 소득 무관")
        else:
            target_info.append(f"- **소득 조건**: 최저 {min_earn_str}원 ~ 최고 {max_earn_str}원")

        
        if meta.get('addAplyQlfcCndCn'): target_info.append(f"- **추가 자격 조건**: {meta.get('addAplyQlfcCndCn')}")
        if target_info: sections["지원 대상 조건"] = target_info

        # 기관 정보
        org_info = []
        if meta.get('sprvsnInstCdNm'): org_info.append(f"- **주관 기관**: {meta.get('sprvsnInstCdNm')}")
        if meta.get('operInstCdNm'): org_info.append(f"- **운영 기관**: {meta.get('operInstCdNm')}")
        if meta.get('refUrlAddr1'): org_info.append(f"- **참고 사이트 1**: {meta.get('refUrlAddr1')}")
        if meta.get('refUrlAddr2'): org_info.append(f"- **참고 사이트 2**: {meta.get('refUrlAddr2')}")
        if org_info: sections["기관 정보"] = org_info
        
        # 만들어진 섹션들을 최종 문자열에 추가
        for title, content_list in sections.items():
            doc_string += f"\n### {title}\n"
            doc_string += "\n".join(content_list)
        
        formatted_strings.append(doc_string)

    return "\n\n".join(formatted_strings)

# =================================================================
# 4. LCEL 체인에 적용하는 방법 (예시)
# =================================================================
# from your_retriever_setup import retriever # 실제 retriever를 가져옵니다.
# from your_prompt_setup import prompt, model, output_parser # 프롬프트, 모델, 파서

# # functools.partial을 사용하여 코드 맵을 함수에 미리 바인딩합니다.
# formatted_docs_func = partial(format_docs_final, code_map=CODE_TABLE_MAP)

# # LCEL 체인 구성
# chain = (
#     {"context": retriever | formatted_docs_func, "question": RunnablePassthrough()}
#     | prompt
#     | model
#     | output_parser
# )

# print("\n✅ LCEL 체인에 적용할 준비가 완료되었습니다.")
# print("   'chain.invoke(\"사용자 질문\")' 형태로 호출하여 사용하세요.")

✅ 엑셀 파일('code_table.xlsx')을 성공적으로 로드하고 파싱했습니다.


In [10]:
format_docs(final_documents[:1], CODE_TABLE_MAP)

'--- [문서 1: 울산웹툰캠퍼스 운영] ---\n\n### 기본 정보\n- **정책 ID**: 20250716005400211297\n- **정책명**: 울산웹툰캠퍼스 운영\n- **정책 요약**: 만화·웹툰 작가, 예비작가를 위한 공간 및 프로그램 운영으로 지역 만화산업 생태계 육성\n- **분류**: 복지문화 > 문화활동\n- **정책 키워드**: 교육지원\n### 지원 내용\n- **상세 지원 내용**: \n○ 사업기간 : ’24. 4.~12. \n\n○ 위    치 : 중구 종가로 406-21 울산비즈파크 6층  \n\n○ 지원대상 : 만화ㆍ웹툰 작가, 웹툰산업 분야 진로 청년 등 \n\n○ 지원내용 : 웹툰 작가 등단을 위한 작품제작 지원(교육‧멘토링) 등\n \n○ 수행기관 : 울산정보산업진흥원\n- **지원 방식**: 프로그램\n- **지원 규모 제한 여부**: 아니오\n### 신청 및 기간\n- **사업 기간**: 2025-04-01 ~ 2025-12-31\n### 지원 대상 조건\n- **학력**: 코드(0049010)\n- **혼인 상태**: 제한없음\n- **직업 상태**: 코드(0013010)\n- **특화 분야**: 코드(0014010)\n- **소득 조건**: 소득 무관\n### 기관 정보\n- **주관 기관**: 울산광역시\n- **운영 기관**: 재단법인울산정보산업진흥원\n- **참고 사이트 1**: https://www.uipa.or.kr/'

In [18]:
from functools import partial
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

# --- 1. 설정값 및 객체 초기화 ---
VDB_DIRECTORY = "./chroma_db_policy"
openai_client = OpenAI()
db_connection_info = {
    'host': 'localhost',
    'database': 'toyprj4',
    'user': 'root',
    'password': '1234'
}
# 실제 DB 커넥션은 체인 내에서 생성 및 종료하여 안정성 확보
def get_db_connection():
    return mysql.connector.connect(**db_connection_info)

# 엑셀 파일로부터 CODE_TABLE_MAP 생성
try:
    df_codes = pd.read_excel('code_table.xlsx', sheet_name='코드정보')
    df_codes['분류'] = df_codes['분류'].ffill()
    CODE_TABLE_MAP = {}
    for _, row in df_codes.iterrows():
        category, code, value = row['분류'], str(row['코드']), row['코드내용']
        if category not in CODE_TABLE_MAP: CODE_TABLE_MAP[category] = {}
        CODE_TABLE_MAP[category][code] = value
    print("✅ 코드 테이블을 성공적으로 로드했습니다.")
except Exception as e:
    print(f"🚨 코드 테이블 로드 오류: {e}")
    CODE_TABLE_MAP = {}


✅ 코드 테이블을 성공적으로 로드했습니다.


In [20]:
# --- R(Retrieval) 파이프라인 ---
retrieval_chain = (
    # 입력: {"query": str}
    # 출력: {"query": str, "filters": dict}
    RunnablePassthrough.assign(
        filters=lambda input_dict: create_filter_from_query(openai_client, input_dict["query"])
    )
    # 출력: {"query": str, "filters": dict, "candidate_ids": list}
    | RunnablePassthrough.assign(
        candidate_ids=lambda input_dict: get_rdb_candidate_ids(get_db_connection(), input_dict["filters"])
    )
    # 출력: {"query": ..., "documents": list[Document]}
    # semantic_search에 필요한 모든 인자를 전달
    | RunnablePassthrough.assign(
        documents=lambda input_dict: semantic_search(
            candidate_ids=input_dict["candidate_ids"],
            original_query=input_dict["query"],
            extracted_filters=input_dict["filters"],
            vdb_directory=VDB_DIRECTORY
        )
    )
)

# --- G(Generation) 파이프라인 ---

# 프롬프트 템플릿
template =  '''
# ROLE
당신은 제공된 [CONTEXT] 문서들을 비판적으로 분석하여 사용자의 [USER'S QUERY]에 가장 정확하고 도움이 되는 답변을 생성하는 '정책 분석 전문 AI'입니다. 당신의 가장 중요한 임무는 관련 없는 정보를 걸러내는 '최종 품질 필터' 역할을 수행하는 것입니다.

# INPUTS
- [CONTEXT]: R(검색) 단계에서 찾아온 하나 이상의 정책 문서 목록입니다.
- [USER'S QUERY]: 사용자의 원본 질문입니다.

# CORE TASK
주어진 [CONTEXT] 문서들을 평가하여 [USER'S QUERY]와 직접적으로 관련된 문서만을 선별하고, 이 선별된 문서들의 내용에만 근거하여 최종 답변을 생성합니다.

# STEP-BY-STEP INSTRUCTIONS
1.  **질문 의도 파악:** 먼저 [USER'S QUERY]를 깊이 있게 분석하여 사용자의 명시적, 암시적 의도를 파악합니다. (예: "주거 대출" -> 주거비용을 '빌리는' 방법에 대한 문의)
2.  **문서 관련성 평가:** [CONTEXT]에 포함된 각 문서를 개별적으로 평가합니다. "이 문서가 사용자의 질문에 직접적으로, 그리고 의미 있게 답변하는가?"를 기준으로 '매우 관련 높음', '약간 관련 있음', '관련 없음'으로 판단하세요.
3.  **정보 선별 및 답변 생성:**
    - **'매우 관련 높음'**으로 판단된 문서들의 내용을 중심으로 답변을 종합합니다.
    - '약간 관련 있음'으로 판단된 문서는, '추가적으로 고려해볼 만한 정책'으로 간략하게만 언급할 수 있습니다.
    - **'관련 없음'**으로 판단된 문서는 답변 생성에 **절대 사용하지 마세요.**
4.  **근거주의 원칙:** 답변의 모든 내용은 반드시 선별된 [CONTEXT] 문서에 명시된 사실에만 근거해야 합니다. 절대 정보를 추측하거나 외부 지식을 활용하지 마세요.
5.  **'정보 없음' 처리:** 만약 평가 결과, 모든 문서가 '관련 없음'으로 판단될 경우, "죄송하지만 문의하신 조건에 정확히 부합하는 정책을 찾지 못했습니다."라고 솔직하게 답변해야 합니다.
6.  **답변 형식:**
    - 사용자가 이해하기 쉽게 친절하고 명확한 문장으로 설명합니다.
    - 여러 정책을 추천할 경우, 각 정책을 번호 목록으로 구분하고 **정책명**과 **핵심 지원 내용**을 명확히 요약하여 제시합니다. 이때, 지원 기간과 운영 기간의 분리를 명확히 하고 둘 다 제시합니다.
    - 정책과 관련된 URL 정보가 있는 경우, URL을 제공하세요.

# --- [매우 중요] 최종 출력 지침 (FINAL OUTPUT INSTRUCTIONS) ---
# 아래 지침은 반드시 엄격하게 준수하여 최종 응답을 생성해야 합니다.
# 1. **사용자에게 직접 말하는 페르소나를 유지하세요.** 절대 자신의 분석 과정("문서를 분석한 결과", "이 문서는 관련성이 높다고 판단됩니다" 등)을 설명하지 마세요.
# 2. **친절한 정책 비서처럼 자연스럽게 답변을 시작하세요.**
# 3. 위에서 지시된 '답변 형식'을 철저히 따라서, 찾아낸 정책 정보를 명확하게 전달하는 데에만 집중하세요.

# CONTEXT
{context}

# USER'S QUERY
{question}

# RESPONSE
'''

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o", temperature=0)
output_parser = StrOutputParser()

# 포맷팅 함수에 코드맵 미리 적용
formatted_docs_func = partial(format_docs, code_map=CODE_TABLE_MAP)

# --- 최종 RAG 체인 결합 ---
final_rag_chain = (
    # 1. 초기 입력을 {"query": user_question} 형태로 만듭니다.
    RunnableLambda(lambda q: {"query": q})
    # 2. 위에서 정의한 3단계의 Retrieval 체인을 실행합니다.
    | retrieval_chain
    # 3. Retrieval 결과를 최종 프롬프트 형식에 맞게 변환합니다.
    | RunnableParallel(
        context=lambda input_dict: formatted_docs_func(input_dict["documents"]),
        question=lambda input_dict: input_dict["query"]
    )
    # 4. 프롬프트, 모델, 파서를 연결하여 최종 답변을 생성합니다.
    | prompt
    | model
    | output_parser
)

# =================================================================
# 3. 완성된 체인 실행
# =================================================================
if __name__ == '__main__':
    user_question = "웹툰 작가가 되고 싶은데, 교육이나 지원금 받을 수 있는 프로그램 찾아줘."
    print(f"--- User Query ---\n{user_question}\n")
    print("--- Executing RAG Chain... ---")
    
    # 최종 체인 실행
    final_response = final_rag_chain.invoke(user_question)
    
    print("\n\n--- Final LLM Response ---")
    print(final_response)


--- User Query ---
웹툰 작가가 되고 싶은데, 교육이나 지원금 받을 수 있는 프로그램 찾아줘.

--- Executing RAG Chain... ---
--- Generated SQL Query (Simplified) ---
SELECT DISTINCT p.policy_id FROM policies p WHERE ((p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))) AND ((p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date)));

--- SQL Parameters ---
[]

--- Generated Vector Search Query ---
웹툰 작가가 되고 싶은데, 교육이나 지원금 받을 수 있는 프로그램 찾아줘. 



--- Final LLM Response ---
웹툰 작가가 되고 싶으시다면, 다음과 같은 프로그램을 고려해보실 수 있습니다:

1. **울산웹툰캠퍼스 운영**
   - **핵심 지원 내용**: 만화·웹툰 작가 및 예비작가를 위한 작품 제작 지원, 교육 및 멘토링 프로그램 제공
   - **운영 기간**: 2025년 4월 1일 ~ 2025년 12월 31일
   - **위치**: 울산 중구 종가로 406-21 울산비즈파크 6층
   - **주관 기관**: 울산광역시
   - **운영 기관**: 재단법인 울산정보산업진흥원
   - **참고 사이트**: [울산정보산업진흥원](https://www.uipa.or.kr/)

이 프로그램은 웹툰 작가로 등단을 목표로 하는 분들에게 작품 제작을 위한 교육과 멘토링을 제공하여 웹툰 산업 분야로의 진출을 지원합니다. 관심이 있으시면 위의 사이트를 방문하여 더 많은 정보를 확인하시고 신청해보세요.
